# Imports and program variables


In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D, MaxPooling1D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
import pandas as pd

from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, Conv1D
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint

from sklearn.model_selection import train_test_split # to split dataset into train and test
from sklearn import preprocessing
from keras.preprocessing.sequence import pad_sequences

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from sklearn import preprocessing

import sys
sys.path.append('../')
from time_series_dataset_loader import TimeSeriesDatasetLoader


Using TensorFlow backend.
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  

In [2]:
def linearize_dataset(dataset):
    ''' Returns a 1 dimensional representation from the dataset '''
    new_dataset = []

    for data_instance in dataset:
        new_dataset_cell = []
        for t_instance in data_instance:
            for value in t_instance[2:]:
                new_dataset_cell.append(value)

        new_dataset.append(new_dataset_cell)
    return np.asarray(new_dataset)
    
    

In [3]:
def preprocessing_data(X, y):
    X = np.asarray(X)
    y = np.asarray(y)
    
    max_len = len(X[0])
    for row in X:
        if len(row) > max_len:
            max_len = len(row)

    X = pad_sequences(X, maxlen=max_len, padding='post', dtype='float64')
    X = linearize_dataset(X)
    return X,y
    

In [45]:
def get_model():
#     model = Sequential()
#     model.add(Conv1D(128, 5,padding='same', input_shape=(504, 504)))
#     model.add(Activation('sigmoid'))
#     model.add(Conv1D(64, 5,padding='same'))
#     model.add(Activation('sigmoid'))
#     model.add(Dropout(0.2))
#     model.add(MaxPooling1D(pool_size=(8)))
#     model.add(Conv1D(32, 5,padding='same',))
#     model.add(Activation('sigmoid'))

#     model.add(Flatten())
#     model.add(Dense(3))
#     model.add(Activation('softmax'))
    model = Sequential()
    model.add(Embedding(1, 504))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='sigmoid'))
    
    return model

In [40]:
a = np.full(100, 2)
b = np.full(2, 100)

In [92]:
POSITIVE_VALUES = np.where(y==1)[0]
NEUTRAL_VALUES = np.where(y==0)[0]
NEGATIVE_VALUES = np.where(y==2)[0][0:600]

In [4]:
def balance_dataset(X, y):
    X = np.asarray(X)
    y = np.asarray(y)
    POSITIVE_VALUES = np.where(y==1)[0]
    NEUTRAL_VALUES = np.where(y==0)[0]
    NEGATIVE_VALUES = np.where(y==2)[0][0:600]
    all_indexes = POSITIVE_VALUES.tolist() + NEUTRAL_VALUES.tolist() + NEGATIVE_VALUES.tolist()
    new_y = np.full(len(POSITIVE_VALUES), 1).tolist() \
        + np.full(len(NEUTRAL_VALUES), 0).tolist() \
        + np.full(len(NEGATIVE_VALUES), 2).tolist()
    
    
    return np.take(X, all_indexes), new_y
    
    
    

In [119]:
np.take(X, all_indexes)

array([array([[ 0.000000e+00,  0.000000e+00,  6.404657e+00, -1.402514e+00,
        -2.236425e+00,  8.194284e-01,  2.850164e+00, -5.242842e+00,
         9.194818e+00,  2.701216e-01, -3.601112e+00, -4.030871e-01,
        -2.343826e+00,  2.297522e+00,  3.568069e+00,  3.482819e-01],
       [ 1.000000e+00,  5.000000e-02,  2.751345e+00, -2.712686e+00,
        -4.506105e+00,  2.210746e+00,  5.707813e+00, -3.487714e+00,
         1.170016e+01,  6.808586e-01,  2.428516e+00,  3.335083e+00,
        -2.458509e+00, -7.786846e-01,  4.941757e+00,  2.185959e+00],
       [ 2.000000e+00,  1.000000e-01, -1.635752e-01, -5.268727e+00,
        -4.959724e+00,  2.915101e+00,  6.304276e+00, -6.310949e-01,
         1.200349e+01,  4.023399e-01,  3.059360e+00,  4.107836e+00,
        -4.426288e+00, -8.730631e-01,  4.143346e+00,  2.863602e+00],
       [ 3.000000e+00,  1.500000e-01, -2.830689e+00, -4.344458e+00,
        -3.053741e+00,  5.715581e+00,  8.206596e+00,  2.755392e+00,
         1.075323e+01,  2.223854e-01, 

In [12]:
DATASET_PATH = '../../datasets/Original/MFCC_5'
# DATASET_PATH = "../../../Output/?MFCC"

## Setting up train and test datasets

In [13]:
dataset_loader = TimeSeriesDatasetLoader(DATASET_PATH)
X, y = dataset_loader.get_dataset(type_='emotion_type')


In [74]:
X, y = balance_dataset(X, y)

In [14]:
X, y = preprocessing_data(X,y)


In [37]:
#from pandas_profiling import ProfileReport
#df = pd.DataFrame(y)
#profile = ProfileReport(df, title='Pandas Profiling Report', html={'style':{'full_width':True}})

In [15]:
X = preprocessing.normalize(X)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)


In [36]:
X[0].shape

(140,)

In [9]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [17]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

def run_classifier(classifier, params, X_train, X_test, y_train, y_test):
    clf = GridSearchCV(classifier, params)
    clf.fit(X_train, y_train)
    return pd.DataFrame(clf.cv_results_)

In [18]:
### SVM ###

params = {'kernel':['linear', 'poly', 'rbf', 'sigmoid'], 'C':[1, 2, 4,5, 6, 8, 10]}
svc = svm.SVC()
run_classifier(svc, params, X_train, X_test, y_train, y_test)

###########

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.918021,0.011732,0.179412,0.002756,1,linear,"{'C': 1, 'kernel': 'linear'}",0.570552,0.576687,0.582822,0.558282,0.549383,0.567545,0.012179,4
1,1.190753,0.006494,0.200017,0.001381,1,poly,"{'C': 1, 'kernel': 'poly'}",0.576687,0.576687,0.570552,0.570552,0.574074,0.573711,0.002750,2
2,1.198328,0.018286,0.204275,0.000383,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.582822,0.576687,0.570552,0.576687,0.567901,0.574930,0.005235,1
3,0.964225,0.101805,0.226343,0.084557,1,sigmoid,"{'C': 1, 'kernel': 'sigmoid'}",0.582822,0.582822,0.570552,0.570552,0.549383,0.571226,0.012223,3
4,1.004677,0.067317,0.186636,0.008729,2,linear,"{'C': 2, 'kernel': 'linear'}",0.496933,0.576687,0.539877,0.552147,0.500000,0.533129,0.030699,13
5,1.201000,0.003233,0.202591,0.000857,2,poly,"{'C': 2, 'kernel': 'poly'}",0.570552,0.521472,0.558282,0.539877,0.561728,0.550382,0.017573,5
6,1.190722,0.007260,0.223443,0.041259,2,rbf,"{'C': 2, 'kernel': 'rbf'}",0.533742,0.576687,0.552147,0.539877,0.506173,0.541725,0.023074,12
7,0.904399,0.032901,0.168921,0.002514,2,sigmoid,"{'C': 2, 'kernel': 'sigmoid'}",0.503067,0.588957,0.564417,0.558282,0.530864,0.549118,0.029526,11
8,0.899812,0.018234,0.174158,0.001787,4,linear,"{'C': 4, 'kernel': 'linear'}",0.435583,0.472393,0.484663,0.527607,0.444444,0.472938,0.032661,24
9,1.246636,0.063273,0.207967,0.011058,4,poly,"{'C': 4, 'kernel': 'poly'}",0.570552,0.521472,0.558282,0.539877,0.555556,0.549148,0.016938,6


In [19]:
### LDA ###
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
params = {'solver':['svd', 'lsqr', 'eigen'], 'shrinkage':[None, 'auto']}
clf = LinearDiscriminantAnalysis()
run_classifier(clf, params, X_train, X_test, y_train, y_test)

###########

/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
numpy.linalg.LinAlgError: the leading minor of order 649 of 'b' is not positive definite. The factorization of 'b' could not be completed and no eigenvalues or eigenvectors were computed.

  FitFailedWarning)
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
numpy.linalg.LinAlgError: the leading minor of order 649 of 'b' is not positive definite. The factorization of 'b' could not be completed and no eigenvalues or eigenvectors were computed.

  FitFailedWarning)
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_shrinkage,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.084682,0.088733,0.002409,0.001058,None,svd,"{'shrinkage': None, 'solver': 'svd'}",0.288344,0.392638,0.368098,0.423313,0.351852,0.364849,0.045217,3
1,5.833917,1.151447,0.004631,0.003137,None,lsqr,"{'shrinkage': None, 'solver': 'lsqr'}",0.429448,0.325153,0.263804,0.392638,0.209877,0.324184,0.080597,4
2,0.591358,0.089739,0.000000,0.000000,None,eigen,"{'shrinkage': None, 'solver': 'eigen'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
3,0.008084,0.004810,0.000000,0.000000,auto,svd,"{'shrinkage': 'auto', 'solver': 'svd'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
4,6.992485,1.727315,0.003526,0.003073,auto,lsqr,"{'shrinkage': 'auto', 'solver': 'lsqr'}",0.472393,0.466258,0.472393,0.447853,0.407407,0.453261,0.024628,1
5,6.889251,0.180846,0.005418,0.003099,auto,eigen,"{'shrinkage': 'auto', 'solver': 'eigen'}",0.472393,0.466258,0.472393,0.447853,0.407407,0.453261,0.024628,1


In [20]:
### KNN ###
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
params = {'n_neighbors':[3,5,7,10,50,100], 'weights':['uniform', 'distance'], 
         'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']}
clf = KNeighborsClassifier()
run_classifier(clf, params, X_train, X_test, y_train, y_test)

###########

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.124693,0.006563,0.591481,0.011307,auto,3,uniform,"{'algorithm': 'auto', 'n_neighbors': 3, 'weigh...",0.417178,0.404908,0.478528,0.404908,0.407407,0.422586,0.028334,45
1,0.138771,0.015725,0.610776,0.027561,auto,3,distance,"{'algorithm': 'auto', 'n_neighbors': 3, 'weigh...",0.417178,0.423313,0.484663,0.460123,0.432099,0.443475,0.025302,37
2,0.109826,0.009184,0.588145,0.025714,auto,5,uniform,"{'algorithm': 'auto', 'n_neighbors': 5, 'weigh...",0.423313,0.429448,0.453988,0.423313,0.432099,0.432432,0.011314,41
3,0.125234,0.013917,0.612433,0.029198,auto,5,distance,"{'algorithm': 'auto', 'n_neighbors': 5, 'weigh...",0.466258,0.429448,0.521472,0.466258,0.438272,0.464341,0.032151,29
4,0.108043,0.005275,0.603692,0.034382,auto,7,uniform,"{'algorithm': 'auto', 'n_neighbors': 7, 'weigh...",0.496933,0.423313,0.509202,0.435583,0.450617,0.463130,0.033959,33
5,0.109389,0.008818,0.560888,0.084907,auto,7,distance,"{'algorithm': 'auto', 'n_neighbors': 7, 'weigh...",0.515337,0.472393,0.527607,0.496933,0.481481,0.498750,0.020514,25
6,0.129217,0.011712,0.614088,0.030427,auto,10,uniform,"{'algorithm': 'auto', 'n_neighbors': 10, 'weig...",0.503067,0.453988,0.576687,0.515337,0.512346,0.512285,0.039091,21
7,0.126126,0.015401,0.604699,0.041975,auto,10,distance,"{'algorithm': 'auto', 'n_neighbors': 10, 'weig...",0.515337,0.478528,0.558282,0.527607,0.493827,0.514716,0.027606,17
8,0.114662,0.010740,0.617581,0.043277,auto,50,uniform,"{'algorithm': 'auto', 'n_neighbors': 50, 'weig...",0.576687,0.576687,0.564417,0.570552,0.567901,0.571249,0.004848,9
9,0.118660,0.015374,0.616354,0.066527,auto,50,distance,"{'algorithm': 'auto', 'n_neighbors': 50, 'weig...",0.576687,0.576687,0.564417,0.570552,0.567901,0.571249,0.004848,9


In [21]:
### Naive Bayes ###
import numpy as np
from sklearn.naive_bayes import GaussianNB
params = {}
clf = GaussianNB()
run_classifier(clf, params, X_train, X_test, y_train, y_test)

###########

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.050721,0.010715,0.016474,0.0032,{},0.165644,0.141104,0.171779,0.159509,0.166667,0.160941,0.010658,1


In [22]:
### Decision Tree ###
import numpy as np
from sklearn.tree import DecisionTreeClassifier
params = {'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random'], 'min_samples_split':[2,4,6,10,20,50,100],
         'min_samples_leaf':[1,2,5,10,20,50,100]}
clf = DecisionTreeClassifier()
run_classifier(clf, params, X_train, X_test, y_train, y_test)

###########

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_min_samples_leaf,param_min_samples_split,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.775285,0.243573,0.002612,0.001410,gini,1,2,best,"{'criterion': 'gini', 'min_samples_leaf': 1, '...",0.509202,0.496933,0.466258,0.478528,0.475309,0.485246,0.015588,88
1,0.200402,0.042105,0.002283,0.001846,gini,1,2,random,"{'criterion': 'gini', 'min_samples_leaf': 1, '...",0.472393,0.503067,0.453988,0.404908,0.493827,0.465637,0.034828,124
2,0.828984,0.167304,0.003966,0.002289,gini,1,4,best,"{'criterion': 'gini', 'min_samples_leaf': 1, '...",0.533742,0.582822,0.423313,0.429448,0.425926,0.479050,0.066557,101
3,0.211651,0.025861,0.003319,0.002500,gini,1,4,random,"{'criterion': 'gini', 'min_samples_leaf': 1, '...",0.386503,0.460123,0.441718,0.404908,0.425926,0.423835,0.026049,189
4,0.773990,0.217166,0.001538,0.000658,gini,1,6,best,"{'criterion': 'gini', 'min_samples_leaf': 1, '...",0.496933,0.478528,0.435583,0.423313,0.438272,0.454525,0.028184,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,0.047766,0.005323,0.002232,0.001473,entropy,100,20,random,"{'criterion': 'entropy', 'min_samples_leaf': 1...",0.576687,0.576687,0.570552,0.570552,0.574074,0.573711,0.002750,1
192,0.205830,0.026781,0.002623,0.001385,entropy,100,50,best,"{'criterion': 'entropy', 'min_samples_leaf': 1...",0.576687,0.576687,0.570552,0.533742,0.574074,0.566349,0.016458,10
193,0.046678,0.004399,0.002715,0.002463,entropy,100,50,random,"{'criterion': 'entropy', 'min_samples_leaf': 1...",0.576687,0.576687,0.503067,0.570552,0.493827,0.544164,0.037509,22
194,0.204022,0.029220,0.002805,0.001619,entropy,100,100,best,"{'criterion': 'entropy', 'min_samples_leaf': 1...",0.576687,0.576687,0.570552,0.533742,0.574074,0.566349,0.016458,10


In [23]:
### MLP classifier ###

import numpy as np
from sklearn.neural_network import MLPClassifier
params = {
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.0001, 0.0005, 0.0020, 0.0050, 0., ] }

clf = MLPClassifier()
run_classifier(clf, params, X_train, X_test, y_train, y_test)

#####################

/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/

/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/

/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/

/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_alpha,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,6.571869,0.884096,0.008951,0.003832,identity,0.0001,lbfgs,"{'activation': 'identity', 'alpha': 0.0001, 's...",0.435583,0.435583,0.460123,0.453988,0.444444,0.445944,0.009825,50
1,11.098583,1.228455,0.010216,0.003759,identity,0.0001,sgd,"{'activation': 'identity', 'alpha': 0.0001, 's...",0.576687,0.576687,0.570552,0.570552,0.574074,0.573711,0.002750,1
2,22.435321,0.685468,0.007613,0.002786,identity,0.0001,adam,"{'activation': 'identity', 'alpha': 0.0001, 's...",0.484663,0.417178,0.484663,0.447853,0.419753,0.450822,0.029653,45
3,7.398101,1.495140,0.013222,0.008197,identity,0.0005,lbfgs,"{'activation': 'identity', 'alpha': 0.0005, 's...",0.453988,0.423313,0.453988,0.460123,0.413580,0.440998,0.018803,56
4,11.059047,1.249738,0.013498,0.001135,identity,0.0005,sgd,"{'activation': 'identity', 'alpha': 0.0005, 's...",0.576687,0.576687,0.570552,0.570552,0.574074,0.573711,0.002750,1
5,19.160743,3.016507,0.005290,0.001598,identity,0.0005,adam,"{'activation': 'identity', 'alpha': 0.0005, 's...",0.484663,0.423313,0.478528,0.429448,0.450617,0.453314,0.024881,37
6,5.649228,1.079807,0.005861,0.002931,identity,0.002,lbfgs,"{'activation': 'identity', 'alpha': 0.002, 'so...",0.478528,0.429448,0.472393,0.453988,0.438272,0.454525,0.018914,35
7,5.665708,0.760709,0.004726,0.000768,identity,0.002,sgd,"{'activation': 'identity', 'alpha': 0.002, 'so...",0.576687,0.576687,0.570552,0.570552,0.574074,0.573711,0.002750,1
8,15.940325,3.069058,0.011694,0.008469,identity,0.002,adam,"{'activation': 'identity', 'alpha': 0.002, 'so...",0.496933,0.441718,0.484663,0.429448,0.432099,0.456972,0.028187,32
9,6.478896,0.934396,0.003456,0.000616,identity,0.005,lbfgs,"{'activation': 'identity', 'alpha': 0.005, 'so...",0.472393,0.404908,0.460123,0.466258,0.419753,0.444687,0.027112,52


In [ ]:
X_traincnn = np.expand_dims(X_train, axis=0)
X_testcnn = np.expand_dims(X_test, axis=0)

In [28]:
X_train = X_train.reshape(1, len(X_train), 504)
X_test = X_test.reshape(1, len(X_test), 504)

In [82]:
embedding_size  = 128
kernel_size = 5
filters = 64
pool_size = 4

lstm_output_size = 70

# Training
batch_size = 64
epochs = 100

model = Sequential()
model.add(Embedding(1, embedding_size, input_length=504))
model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(3))
model.add(Activation('sigmoid'))


from keras.optimizers import SGD
opt = SGD(lr=0.01)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])
cnnhistory=model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test))



Train on 998 samples, validate on 428 samples
Epoch 1/100
998/998 [==============================] - 7s 7ms/step - loss: 1.1022 - accuracy: 0.2275 - val_loss: 1.0966 - val_accuracy: 0.4112
Epoch 2/100
998/998 [==============================] - 6s 6ms/step - loss: 1.0964 - accuracy: 0.4248 - val_loss: 1.0918 - val_accuracy: 0.4112
Epoch 3/100
998/998 [==============================] - 6s 6ms/step - loss: 1.0912 - accuracy: 0.4248 - val_loss: 1.0875 - val_accuracy: 0.4112
Epoch 4/100
998/998 [==============================] - 6s 6ms/step - loss: 1.0864 - accuracy: 0.4248 - val_loss: 1.0836 - val_accuracy: 0.4112
Epoch 5/100
998/998 [==============================] - 6s 6ms/step - loss: 1.0823 - accuracy: 0.4248 - val_loss: 1.0799 - val_accuracy: 0.4112
Epoch 6/100
998/998 [==============================] - 6s 6ms/step - loss: 1.0784 - accuracy: 0.4248 - val_loss: 1.0765 - val_accuracy: 0.4112
Epoch 7/100
998/998 [==============================] - 7s 7ms/step - loss: 1.0748 - accuracy: 0.

In [ ]:
batch_size = 256
epochs = 500
from keras.optimizers import SGD
opt = SGD(lr=0.01)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = opt, metrics=['accuracy'])
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=20, min_lr=0.0000001)
mcp_save = ModelCheckpoint('../model_checkpoints/two_split.h5', save_best_only=True, monitor='val_loss', mode='min')

In [49]:
cnnhistory=model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test))
# , callbacks=[mcp_save, lr_reduce]

ValueError: Error when checking target: expected dense_5 to have shape (1,) but got array with shape (3,)

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 504, 128)          768       
_________________________________________________________________
activation_1 (Activation)    (None, 504, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 504, 64)           41024     
_________________________________________________________________
activation_2 (Activation)    (None, 504, 64)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 504, 64)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 63, 64)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 63, 32)           

array([[[  6.404657  ,  -1.402514  ,  -2.236425  , ...,   0.        ,
           0.        ,   0.        ],
        [  0.8584499 ,  -0.06234932,   0.6381655 , ...,   0.        ,
           0.        ,   0.        ],
        [  5.846846  , -10.07883   , -11.02912   , ...,   0.        ,
           0.        ,   0.        ],
        ...,
        [ 12.99762   ,  -7.800581  ,  -1.155792  , ...,  -0.5558    ,
          -1.747546  ,   3.087212  ],
        [  3.280588  ,   2.007889  ,  -2.019495  , ...,  -0.9734643 ,
           0.8074539 ,   2.71006   ],
        [  5.084667  ,  -3.900648  ,  12.28177   , ...,  -0.7672844 ,
           2.181755  ,   4.326027  ]]])

In [ ]:
''' More Preprocessing '''

In [129]:
from sklearn.preprocessing import RobustScaler

scalers = {}
for i in range(X.shape[1]):
    scalers[i] = RobustScaler()
    X[:, i, :] = scalers[i].fit_transform(X[:, i, :]) 


In [16]:
from sklearn.preprocessing import Normalizer

scalers = {}
for i in range(X.shape[1]):
    scalers[i] = Normalizer()
    X[:, i, :] = scalers[i].fit_transform(X[:, i, :]) 

In [18]:
X[0][0]

array([ 0.        ,  0.        ,  0.28221284,  0.08020835, -0.09187018,
       -0.10788761,  0.62792463,  0.3675004 ,  0.16599244,  0.18636285,
       -0.35420188,  0.19850305, -0.14629544, -0.18601133, -0.21646694,
        0.18571278])

In [16]:
X_train.reshape(1)

ValueError: cannot reshape array of size 502992 into shape (1,)

In [ ]:
''' Finishing Preprocessing '''

In [ ]:
# See X and y details
print(X[:2])
print(X.shape)

print(y[:3])
print(y.shape)

In [41]:
# Split the dataset in train and test

print("\nX_train:\n")
print(X_train[0][0])
print(X_train.shape)

print("\nX_test:\n")
print(X_test[0][0])
print(X_test.shape)

print("\ny_train:\n")
print(y_train[:3])
print(y_train.shape)

print("\ny_test:\n")
print(y_test[:3])
print(y_test.shape)


X_train:

9.105775999999999
(1364, 504)

X_test:

-0.1714754
(585, 504)

y_train:

[2 1 2]
(1364,)

y_test:

[1 2 2]
(585,)


In [42]:
# Create categorical matrices



In [33]:
num_rows = X[0].shape[0]
num_columns = X[0].shape[1]
num_channels = 1

X_train = X_train.reshape(X_train.shape[0], num_rows, num_columns, num_channels)
X_test = X_test.reshape(X_test.shape[0], num_rows, num_columns, num_channels)

print("\nX_train:\n")
print(X_train[0][0])
print(X_train.shape)

print("\nX_test:\n")
print(X_test[0][0])
print(X_test.shape)

IndexError: tuple index out of range

## Building and Compiling Model

In [46]:
# top-k category accuracy
import functools
import keras
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',top3_acc])

In [47]:
# Define bath and epochs
batch_size = 256
epochs = 400
checkpoint_file = '../models/model_checkpoints/original_window_2.h5'
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',top3_acc])
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=20, min_lr=0.0000001)
mcp_save = ModelCheckpoint('../models/model_checkpoints/two_split.h5', save_best_only=True, monitor='val_loss', mode='min')
cnnhistory=model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test), callbacks=[mcp_save, lr_reduce])

ValueError: Error when checking input: expected conv1d_3_input to have 3 dimensions, but got array with shape (1364, 504)

In [25]:
accuracy_list = cnnhistory.history['val_accuracy']
highest_index = cnnhistory.history['val_accuracy'].index(np.sort(cnnhistory.history['val_accuracy'])[-1])
print(cnnhistory.history['val_accuracy'][highest_index])

0.6239316463470459


In [ ]:
# without prepocessing: 0.5863248109817505
# robust preprocessing: 0.6034188270568848
# normalization: 0.6239316463470459

In [ ]:
import matplotlib.pyplot as plt

# Model Summary

model.summary()

# PLT History info
plt.plot(cnnhistory.history['loss'])
plt.plot(cnnhistory.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()